In [1]:
import sys
sys.path.append('/work/utilis')

from lib_to_use import *
from general_functions import *
from paths import *

/root/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Torch version:  2.6.0+cu124
Pyterrier version:  0.13.0
/work/utilis/lib_to_use.py:32: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
# Carica i dati
with open(val_processed_path, 'rb') as f:
    df_val = pickle.load(f)
qrels_val = load_qrels(qrels_val_path)
qrels_val_df = qrels_to_df(qrels_val)

# Carica l'indice
index_ref = pt.IndexFactory.of("./pyterrier_index")

Java started (triggered by IndexFactory.of) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
# Crea pipeline con espansione
expander = T5QueryExpander(max_tokens=10)
bm25_T5 = create_experiment_pipeline(index_ref, expander, num_results=100)

Device set to use cpu


In [4]:

# Esegui e valuta
start_time = time.time()
expanded_results = bm25_T5.transform(df_val[['qid', 'query']])
print("Time taken to retrieve expanded queries:", time.time() - start_time)
expanded_results

Time taken to retrieve expanded queries: 52.761446952819824


,qid,docid,docno,text,rank,score,query
0,academia_143743,66,academia_28111,mean fellowship phdstudent extern award ok lis...,0,24.197326,answer question one ask histor research came a...
1,academia_143743,14,academia_12035,short answer free authorship also known gift a...,1,23.709126,answer question one ask histor research came a...
2,academia_143743,147,academia_76526,conceptu pointsfor two year ive work draft pap...,2,21.791199,answer question one ask histor research came a...
3,academia_143743,3441,law_5712,first law order taken accur depict new york tr...,3,21.178890,answer question one ask histor research came a...
4,academia_143743,6525,scifi_117801,question specif ask canon scifi fantasi im qui...,4,18.967244,answer question one ask histor research came a...
...,...,...,...,...,...,...,...
10195,politics_53742,3946,money_80511,youv alreadi got emerg save suffici need agre ...,95,8.111755,u polic treat use counterfeit money emerg one ...
10196,politics_53742,3403,law_27889,go vari base jurisdictionin wisconsin attempt ...,96,8.051337,u polic treat use counterfeit money emerg one ...
10197,politics_53742,3390,law_29132,cite articl thank interviewe provid scan polic...,97,8.039932,u polic treat use counterfeit money emerg one ...
10198,politics_53742,3780,money_19075,simpli pay currenc regist market normal nativ ...,98,8.025304,u polic treat use counterfeit money emerg one ...


In [5]:
# Per valutare i risultati BM25
eval_bm25_T5 = pt.Evaluate(expanded_results, qrels_val_df, 
                          metrics=["P_1", "recall_100", "map_cut_100", "ndcg_cut_3"])
print("BM25 + T5 query expansion Evaluation:", eval_bm25_T5)
with open("/work/PIR_data_unzip/PIR_data/answer_retrieval/saved_results/bm25_T5_results.pkl", "wb") as f:
    pickle.dump(expanded_results, f)

BM25 + T5 query expansion Evaluation: {'P_1': 0.7142857142857143, 'recall_100': 0.9285714285714286, 'map_cut_100': 0.7731338936073157, 'ndcg_cut_3': 0.7722282426749296}


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=144ec01f-394f-474f-b507-b786ab13b472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>